# COGS 118B - Final Project

# Insert title here

## Group members

- Pelé
- Diego Maradonna
- Johan Cruyff
- Roberto Carlos
- Franz Beckenbaur

# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- what your goal/problem is
- what the data used represents 
- the solution/what you did
- major results you came up with (mention how results are measured) 

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 2 or 3 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data

Detail how/where you obtained the data and cleaned it (if necessary)

If the data cleaning process is very long (e.g., elaborate text processing) consider describing it briefly here in text, and moving the actual clearning process to another notebook in your repo (include a link here!).  The idea behind this approach: this is a report, and if you blow up the flow of the report to include a lot of code it makes it hard to read.

Please give the following infomration for each dataset you are using
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc you have done should be demonstrated here!


- [Dataset 1](https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks): This dataset contains the stock prices of the S&P 500 companies. The dataset contains 5 columns and 6068010 rows. The columns are: `date`, `name`, `open`, `high`, `low`, `close`, `volume`. The `date` column is a string that represents the date of the stock price. The `name` column is a string that represents the name of the company. The `open`, `high`, `low`, `close`, and `volume` columns are floats that represent the stock price of the company. The dataset was cleaned by removing any rows with missing values.
- [Dataset 2](https://pypi.org/project/yfinance/): This is a Python library that allows us to download the stock prices of companies. The library was used to download various other metrics of the S&P 500 companies. The library contains a (usually numerical) value for each metric, with hundreds of metrics for each company

In [16]:
%pip install yfinance
%pip install matplotlib
%pip install scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 27.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 7.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 5.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score


In [6]:
# Read the S&P 500 companies data from the CSV file
sp500_companies = pd.read_csv('sp500_companies.csv')

# Define lists to store data
symbols = []
sectors = []
current_prices = []
market_caps = []
volatilities = []
pe_ratios = []
dividend_yields = []

# Loop through each company
for index, row in sp500_companies.iterrows():
    try:
        symbol = row['Symbol']
        # Fetch data for the company
        company_data = yf.Ticker(symbol)
        
        # Get company info
        info = company_data.info
        
        # Append data to lists
        symbols.append(symbol)
        sectors.append(company_data.info['sector'])
        current_prices.append(row['Currentprice'])
        market_caps.append(row['Marketcap'])
        volatilities.append(company_data.history(period="1y").Close.pct_change().std())
        pe_ratios.append(info.get('forwardPE', None))
        dividend_yields.append(info.get('dividendYield', None))
    except:
        print(f"Data for {symbol} not found.")

# Create DataFrame
data = {
    'Symbol': symbols,
    'Sectors': sectors,
    'Current Price': current_prices,
    'Market Cap': market_caps,
    'Volatility': volatilities,
    'P/E Ratio': pe_ratios,
    'Dividend Yield': dividend_yields
}
df = pd.DataFrame(data)

df = pd.get_dummies(df, columns=['Sectors'])

# Export the DataFrame to a CSV file
df.to_csv('sp500_stock_data_with_metrics.csv', index=False)

# Display the DataFrame
print(df)

    Symbol  Current Price     Market Cap  Volatility  P/E Ratio  \
0     MSFT         402.65  2991866511360    0.014345  30.138474   
1     AAPL         170.12  2626975956992    0.012051  23.726637   
2     NVDA         859.64  2149100027904    0.029335  28.837303   
3     AMZN         174.12  1808654008320    0.019152  32.790960   
4     GOOG         133.78  1657520783360    0.017612  17.129322   
..     ...            ...            ...         ...        ...   
497    CMA          51.74     6824402944    0.038637   8.358644   
498   GNRC         110.09     6635046912    0.032503  13.744069   
499    VFC          15.82     6151069184    0.035928  11.632353   
500   ZION          40.95     6066947584    0.040845   8.825432   
501    WHR         103.13     5635415040    0.021676   6.907569   

     Dividend Yield  Sectors_Basic Materials  Sectors_Communication Services  \
0            0.0072                    False                           False   
1            0.0053                

Silhouette Score (Gaussian Mixture Model, 2 clusters): 0.17620038735694185
Silhouette Score (Gaussian Mixture Model, 3 clusters): 0.16767693915151394
Silhouette Score (Gaussian Mixture Model, 4 clusters): 0.18151734220598742
Silhouette Score (Gaussian Mixture Model, 5 clusters): 0.2411052399503303
Silhouette Score (Gaussian Mixture Model, 6 clusters): 0.29795472738580997
Silhouette Score (Gaussian Mixture Model, 7 clusters): 0.3433870331979371
Silhouette Score (Gaussian Mixture Model, 8 clusters): 0.4017241137843583
Silhouette Score (Gaussian Mixture Model, 9 clusters): 0.4431450489339882
Silhouette Score (Gaussian Mixture Model, 10 clusters): 0.49457953963115064
Silhouette Score (Gaussian Mixture Model, 11 clusters): 0.5549204176772427
Silhouette Score (Gaussian Mixture Model, 12 clusters): 0.5708409876051428
Silhouette Score (Gaussian Mixture Model, 13 clusters): 0.5727091499327581
Silhouette Score (Gaussian Mixture Model, 14 clusters): 0.5268391402370031
Silhouette Score (Gaussian M

# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?



# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

### Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination.

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Footnotes
<a name="lorenznote"></a>1.[^](#lorenz): Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html<br> 
<a name="admonishnote"></a>2.[^](#admonish): Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.<br>
<a name="sotanote"></a>3.[^](#sota): Perhaps the current state of the art solution such as you see on [Papers with code](https://paperswithcode.com/sota). Or maybe not SOTA, but rather a standard textbook/Kaggle solution to this kind of problem
